# 0.1 实验说明

- 搭建 DVWA 靶机，完成其中的暴力破解（Brute Force）实验 
- 将实验内容填写在给定 Jupyter Notebook 文件的指定位置即可
- 需提交材料
    - CG平台：Jupyter Notebook 文件（带输出结果），还需提交一份相同内容的 PDF 格式文件
    - 相同内容的纸质打印版
- 实验内容会有查重，如发现问题，会现场验收

# 0.2 学生信息

- 班级：网络21-1班
- 学号：2021083111  
- 姓名：李修涵

# 1. Low 级别

## 1.1 问题分析

由于没有对登录页面进行相关的防暴力破解机制，如无验证码、有验证码但验证码未在服务器端校验以及无登录错误次数限制等，导致攻击者可通过暴力破解获取用户登录账户及口令，从而获取网站登录访问权限


## 1.2 防护情况


就是一个只实现了登入的代码，只是对密码进行md5加密，防止我们通过密码进行SQL注入（可以使用username进行SQL注入），可没有对帐号的登入尝试次数做限制。

## 1.3 攻击源码

打开burp对此网站抓包，随便输入用户名和密码

 <img src='./dvwa-BF/1.png' width="40%">

然后进行抓包

 <img src='./dvwa-BF/2.png' width="40%">

将此数据包发送到intruder模块，这里会有几个标出来的字段，直接选择clear，选中需要爆破是username和password的值利用add添加.
 <img src='./dvwa-BF/3.png' width="40%">

选中paysloads模块，进行设置，直接在payload options选择load去添加字典，或者用add添加单个字段

 <img src='./dvwa-BF/4.png' width="40%">

输出结果

 <img src='./dvwa-BF/13.png' width="40%">

# 2. Medium 级别

## 2.1 问题分析
没有验证码，错误次数限制，而且用户名密码明文传输，但是增加了登陆间隔时间，所以用 burpsuit 抓包后用字典自动化爆破

## 2.2 防护情况

这边的代码加上了一个，延迟两秒才能下一个，所以延长了爆破时间，增加的是时间成本。

## 2.3 攻击源码


 <img src='./dvwa-BF/3.png' width="40%">

需要设置时停

 <img src='./dvwa-BF/7.png' width="20%">

输出结果

<img src = "./dvwa-BF/9.png" width = "30%">


# 3. High 级别

## 3.1 问题分析

In [ ]:
if( isset( $_GET[ 'Login' ] ) ) {
    // Check Anti-CSRF token
    checkToken( $_REQUEST[ 'user_token' ], $_SESSION[ 'session_token' ], 'index.php' );

    // Sanitise username input
    $user = $_GET[ 'username' ];
    $user = stripslashes( $user );
    $user = ((isset($GLOBALS["___mysqli_ston"]) && is_object($GLOBALS["___mysqli_ston"])) ? mysqli_real_escape_string($GLOBALS["___mysqli_ston"],  $user ) : ((trigger_error("[MySQLConverterToo] Fix the mysql_escape_string() call! This code does not work.", E_USER_ERROR)) ? "" : ""));

    // Sanitise password input
    $pass = $_GET[ 'password' ];
    $pass = stripslashes( $pass );
    $pass = ((isset($GLOBALS["___mysqli_ston"]) && is_object($GLOBALS["___mysqli_ston"])) ? mysqli_real_escape_string($GLOBALS["___mysqli_ston"],  $pass ) : ((trigger_error("[MySQLConverterToo] Fix the mysql_escape_string() call! This code does not work.", E_USER_ERROR)) ? "" : ""));
    $pass = md5( $pass );

    // Check database
    $query  = "SELECT * FROM `users` WHERE user = '$user' AND password = '$pass';";
    $result = mysqli_query($GLOBALS["___mysqli_ston"],  $query ) or die( '<pre>' . ((is_object($GLOBALS["___mysqli_ston"])) ? mysqli_error($GLOBALS["___mysqli_ston"]) : (($___mysqli_res = mysqli_connect_error()) ? $___mysqli_res : false)) . '</pre>' );

    if( $result && mysqli_num_rows( $result ) == 1 ) {
        // Get users details
        $row    = mysqli_fetch_assoc( $result );
        $avatar = $row["avatar"];

        // Login successful
        echo "<p>Welcome to the password protected area {$user}</p>";
        echo "<img src=\"{$avatar}\" />";
    }
    else {
        // Login failed
        sleep( rand( 0, 3 ) );
        echo "<pre><br />Username and/or password incorrect.</pre>";
    }

    ((is_null($___mysqli_res = mysqli_close($GLOBALS["___mysqli_ston"]))) ? false : $___mysqli_res);
}

// Generate Anti-CSRF token
generateSessionToken();



## 3.2 防护情况

前面的防护基础上加上了Anti-CSRF token来抵御CSRF的攻击，使用了stripslashes函数和mysqli_real_esacpe_string来抵御SQL注入和XSS的攻击。由于使用了Anti-CSRF token，每次服务器返回的登陆页面中都会包含一个随机的user_token的值，用户每次登录时都要将user_token一起提交。服务器收到请求后，会优先做token的检查，再进行sql查询。

## 3.3 攻击源码

修改options下的grep-extract

 <img src='./dvwa-BF/10.png' width="30%">

找到token，先修改值，在单击refetch response

 <img src='./dvwa-BF/11.png' width="30%">
 <br>
 <img src='./dvwa-BF/12.png' width="30%">

将payloads下的payload set 改为2

输出结果

 <img src='./dvwa-BF/14.png' width="30%">



# 4. Impossible 级别分析

在上一难度的基础上对用户的操作做了限制，当登入失败3次后，账号会锁住15s，同时采用了更为安全的PDO（PHP Data Object）机制防御sql注入，这里因为不能使用PDO扩展本身执行任何数据库操作，而sql注入的关键就是通过破坏sql语句结构执行恶意的sql命令。


# 5. 心得体会

理解攻击原理：通过尝试不同难度级别的Brute Force攻击，我更深入地理解了Brute Force攻击的原理和工作方式。这包括如何选择合适的字典、暴力破解的原理等。
认识到不同难度的差异：通过破解不同难度级别的目标，我意识到不同难度级别之间存在着差异。比如，低难度级别可能存在较为简单的密码，而高难度级别可能会有更复杂的防御措施。
强调安全意识：通过这个过程，我更加重视密码安全和防御措施的重要性。这可能会促使我在日常生活和工作中更加注意密码的选择和管理，以及其他安全实践。
练习技能：Brute Force攻击是网络安全领域的一项基本技能。通过在DVWA靶场上的练习，我可以提高自己的技能水平，为应对实际场景中的安全挑战做好准备。
学习防御措施：在尝试破解不同难度级别的目标时，我也意识到了一些防御措施的重要性。这可能包括强化密码策略、实施账户锁定机制、使用多因素认证等等。

# 6. 参考文献

1. dvwa靶场Brute Force（暴力破解）全难度教程（附代码分析）,https://blog.csdn.net/m0_73248913/article/details/128769294
2. DVWA —— Brute Force 暴力破解, https://blog.csdn.net/YouTheFreedom/article/details/130786146
3. DVWA实战篇- Brute Force（暴力破解），https://zhuanlan.zhihu.com/p/378191004

# 7. 评分及评语

- 评分：
- 评语：